In [6]:
import pymc as pm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ipywidgets import interact, FloatSlider

def calc_prior_dist(s_a, s_b, s_g, a_bar_mu, a_bar_sigma):
    with pm.Model() as model:
        # Global Intercept and standard deviation for Modules
        a_bar = pm.Normal('a_bar', mu=a_bar_mu, sigma=a_bar_sigma)
        
        # Standard Deviations for Modules, Parameters and Interactions
        sigma_a = pm.Exponential('sigma_a', s_a)
        sigma_b = pm.Exponential('sigma_b', s_b)
        sigma_g = pm.Exponential('sigma_g', s_g)
        
        # Non-centered parameterizations
        a_offset = pm.Normal('a_offset', mu=0, sigma=1)
        a_m = pm.Deterministic('a_m', a_bar + sigma_a * a_offset)

        b_offset = pm.Normal('b_offset', mu=0, sigma=1)
        b_p = pm.Deterministic('b_p', sigma_b * b_offset)

        g_offset = pm.Normal('g_offset', mu=0, sigma=1)
        g_mp = pm.Deterministic('g_mp', sigma_g * g_offset)


        # Link function (logit), unbounded to (0,1) probability
        mu_before = pm.Deterministic('mu_before', a_m + b_p + g_mp)
        mu = pm.Deterministic('mu', pm.math.exp(a_m + b_p + g_mp))
        
        # Beta distribution likelihood 
        theta = pm.Uniform('theta', 0.0, 100)
        
        # Sample from the model
        idata = pm.sample_prior_predictive(samples=3000)



    # Fetch and flatten priors
    prior_a_bar = idata.prior['a_bar'].values.flatten()
    prior_sigma_a = idata.prior['sigma_a'].values.flatten()
    prior_a_m = idata.prior['a_m'].values.flatten()
    prior_sigma_b = idata.prior['sigma_b'].values.flatten()
    prior_b_p = idata.prior['b_p'].values.flatten()
    prior_sigma_g = idata.prior['sigma_g'].values.flatten()
    prior_g_mp = idata.prior['g_mp'].values.flatten()
    prior_mu= idata.prior['mu'].values.flatten()
    prior_mu_before= idata.prior['mu_before'].values.flatten()
    prior_theta = idata.prior['theta'].values.flatten()

    #simulated_observations = np.random.beta(a=prior_p * prior_theta, b=(1 - prior_p) * prior_theta)
    print(prior_mu[0:10])
    

    simulated_observations = np.random.negative_binomial(n=prior_theta, p=(prior_theta)/(prior_mu + prior_theta))
    
    #Predicated observations
    plt.figure(figsize=(4, 3))
    sns.histplot(simulated_observations, color='darkcyan', binrange=(0, 7000))
    plt.title('Simulated Observations of Coverage using priors ')
    plt.xlabel('Values')
    plt.ylabel('Density')
    plt.show()

    plt.figure(figsize=(20, 15))

    # p
    plt.subplot(5, 2, 1)
    sns.histplot(prior_mu_before, color='k', )
    plt.title('Prior Distribution of p - Logistic / Inverse Logit')
    plt.xlabel('p')
    plt.ylabel('Density')

    # p
    plt.subplot(5, 2, 2)
    sns.histplot(prior_mu, color='purple', binrange=(0, 10000))
    plt.title('Prior Distribution of p - Logistic / Inverse Logit')
    plt.xlabel('p')
    plt.ylabel('Density')

    # theta
    plt.subplot(5, 2, 3)
    sns.histplot(prior_theta, color='orange')
    plt.title('Prior Distribution of theta - Unifrom(10,200)')
    plt.xlabel('p_before')
    plt.ylabel('Density')


    # a_bar
    plt.subplot(5, 2, 4)
    sns.histplot(prior_a_bar, color='blue')
    plt.title('Prior Distribution of a_bar - Normal(0, 1.5)')
    plt.xlabel('a_bar')
    plt.ylabel('Density')

    # sigma
    plt.subplot(5, 2, 5)
    sns.histplot(prior_sigma_a, color='red')
    plt.title('Prior Distribution of sigma - Exponential(1.5)')
    plt.xlabel('sigma')
    plt.ylabel('Density')

    # a_m
    plt.subplot(5, 2, 6)
    sns.histplot(prior_a_m, color='green')
    plt.title('Prior Distribution of a_m - Normal(a_bar, sigma)')
    plt.xlabel('a_m')
    plt.ylabel('Density')

    # sigma
    plt.subplot(5, 2, 7)
    sns.histplot(prior_sigma_b, color='salmon')
    plt.title('Prior Distribution of sigma - Exponential(1.5)')
    plt.xlabel('sigma')
    plt.ylabel('Density')

    # b_p
    plt.subplot(5, 2, 8)
    sns.histplot(prior_b_p, color='pink')
    plt.title('Prior Distribution of b_p - Normal(a_bar, sigma)')
    plt.xlabel('b_p')
    plt.ylabel('Density')

    # sigma
    plt.subplot(5, 2, 9)
    sns.histplot(prior_sigma_g, color='brown')
    plt.title('Prior Distribution of sigma - Exponential(1.5)')
    plt.xlabel('sigma')
    plt.ylabel('Density')

    # g_mp
    plt.subplot(5, 2, 10)
    sns.histplot(prior_g_mp, color='yellow')
    plt.title('Prior Distribution of b_p - Normal(a_bar, sigma)')
    plt.xlabel('g_mp')
    plt.ylabel('Density')

    

    plt.tight_layout()
    plt.show()

    


s_a = FloatSlider(value=4, min=0.5, max=6, step=0.1, description='Sigma_alpha (sigma_a):')
s_b = FloatSlider(value=4, min=0.5, max=6, step=0.1, description='Sigma_beta (sigma_b):')
s_g = FloatSlider(value=4, min=0.5, max=6, step=0.1, description='Sigma_gamma (sigma_g):')
a_bar_mu = FloatSlider(value=7, min=0, max=10, step=0.1, description='alpha bar mu (a_bar_mu):')
a_bar_sigma = FloatSlider(value=0.5, min=0, max=3, step=0.1, description='alpha bar sigma (a_bar_sigma):')



interact(calc_prior_dist, s_a=s_a, s_b=s_b, s_g=s_g, a_bar_mu=a_bar_mu, a_bar_sigma=a_bar_sigma)

interactive(children=(FloatSlider(value=4.0, description='Sigma_alpha (sigma_a):', max=6.0, min=0.5), FloatSli…

<function __main__.calc_prior_dist(s_a, s_b, s_g, a_bar_mu, a_bar_sigma)>

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def plot_negative_binomial(log_mu, alpha):
    # Calculate mu from the log_mu (linear predictor)
    mu = np.exp(log_mu)
    
    # Convert PyMC-style parameters to NumPy-style parameters
    n = alpha
    p = alpha / (mu + alpha)
    
    # Generate samples
    samples = np.random.negative_binomial(n, p, size=10000)
    
    # Plot the histogram of the samples
    plt.figure(figsize=(10, 6))
    plt.hist(samples, bins=50, alpha=0.75, color='blue', edgecolor='black', density=True)
    plt.title(f"Negative Binomial Distribution\nLog Mean = {log_mu}, Mean = {round(mu, 2)}, Dispersion = {alpha}")
    plt.xlabel("Value")
    plt.ylabel("Density")
    plt.grid(True)
    plt.show()

# Create interactive widgets
log_mu_slider = FloatSlider(value=np.log(1500), min=np.log(100), max=np.log(5000), step=0.1, description='Log Mean (log_mu):')
alpha_slider = FloatSlider(value=9, min=1, max=30, step=1, description='Dispersion (alpha):')

interact(plot_negative_binomial, log_mu=log_mu_slider, alpha=alpha_slider)


interactive(children=(FloatSlider(value=7.313220387090301, description='Log Mean (log_mu):', max=8.51719319141…

<function __main__.plot_negative_binomial(log_mu, alpha)>

In [2]:
import formulaic
import pymc as pm
import pandas as pd

data = pd.read_csv("../single_parameter/combined_data/statistics.csv")


# Dummy variables for Module and Parameters
model_formula = 'AlgorithmIterations ~ 0 + C(TargetModule) + C(TuningParameters, contr.treatment("NONE"))'
design_matrix = formulaic.model_matrix(model_formula, data=data)

module_matrix = design_matrix.rhs.iloc[:, :24]
parameter_matrix = design_matrix.rhs.iloc[:, 24:]

# Dummy variables for interaction terms
model_formula = 'AlgorithmIterations ~ 0 + C(TargetModule) : C(TuningParameters)'
design_matrix = formulaic.model_matrix(model_formula, data=data)

# Filter out columns that contain 'T.NONE' in their name
columns_to_drop = [col for col in design_matrix.rhs.columns if 'T.NONE' in col]

# Drop the identified columns
design_matrix.rhs.drop(columns=columns_to_drop, axis=1, inplace=True)
interaction_matrix = design_matrix.rhs.iloc[:,:]
design_matrix.lhs.describe()


,AlgorithmIterations
count,9360.000000
mean,1514.810043
std,696.050921
min,289.000000
25%,1014.750000
50%,1387.000000
75%,1786.000000
max,4039.000000


In [3]:
with pm.Model() as model:
    # Global Intercept and standard deviation for Modules
    a_bar = pm.Normal('a_bar', mu=7, sigma=0.5)
    
    # Standard Deviations for Modules, Parameters and Interactions
    sigma_a = pm.Exponential('sigma_a', 4.0)
    sigma_b = pm.Exponential('sigma_b', 4.0)
    sigma_g = pm.Exponential('sigma_g', 4.0)
    
    # Non-centered parameterizations
    a_offset = pm.Normal('a_offset', mu=0, sigma=1, shape=24)
    a_m = pm.Deterministic('a_m', a_bar + sigma_a * a_offset)

    b_offset = pm.Normal('b_offset', mu=0, sigma=1, shape=12)
    b_p = pm.Deterministic('b_p', sigma_b * b_offset)

    g_offset = pm.Normal('g_offset', mu=0, sigma=1, shape=288)
    g_mp = pm.Deterministic('g_mp', sigma_g * g_offset)

    
    # Activate the correct dummy variables
    log_a = pm.math.dot(module_matrix, a_m)
    log_b = pm.math.dot(parameter_matrix, b_p)
    log_g = pm.math.dot(interaction_matrix, g_mp)

    # Link function (log)
    mu = pm.Deterministic('mu', pm.math.exp(log_a + log_b + log_g))
    
    # Beta distribution likelihood 
    theta = pm.Uniform('theta', 0, 100)
    Y_obs = pm.NegativeBinomial('Y_obs', n=theta, p=(theta)/(mu + theta), observed=design_matrix.lhs['AlgorithmIterations'])
    
    # Sample from the model
    trace = pm.sample(1000, chains=4,return_inferencedata=True, progressbar=True, target_accept=0.95)
    log_lik = pm.compute_log_likelihood(trace)
    
print("Model building complete.")

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a_bar, sigma_a, sigma_b, a_offset, b_offset, theta]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1365 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

Model building complete.


In [4]:
import arviz as az

az.to_netcdf(trace, "model_overhead_trace.nc")

'model_overhead_trace.nc'